In [1]:
import os
import sys
import boto
import ast
from datetime import datetime
HOME = os.environ.get('HOME')
sys.path.append(HOME + "/comscore_processing")
from utils.utils import apply_process_row, get_log_date,process_row
from utils.log_processors import UserProcessor
#from RDLogging.logging import RDLogger
sc.addPyFile(HOME + "/comscore_processing/utils.zip")
sc.addPyFile(HOME + "/comscore_processing/RDLogging.zip")
from utils.constants import *
from utils.utils import get_date_range,eval_dict
from utils.session_aggregators import SessionAggregator
from utils.dynamo_writer import record_log_date
from utils.category_counts import process_category_combos
from boto.dynamodb2.layer1 import DynamoDBConnection
from utils.user_ids_by_site import write_user_ids_by_site
from utils.parquet_writer import ParquetWriter,VISITOR_SCHEMA
from boto.s3.connection import S3Connection
from boto.s3.connection import OrdinaryCallingFormat
from operator import add
from utils.category_counts import site_combos_to_process
import csv, io
from urllib.parse import urlparse,parse_qs
import argparse
from utils.config import config
from utils.utils import (
        S3_FIRST_PASS_PATTERN, del_s3_dir, RDMSR_BUCKET,
        get_date_range, parser, eval_dict,
        combineLists,period_delta)

In [2]:
data = sc.textFile(S3_FILE_PATTERN.format('20160910'))
header = sc.broadcast(data.first().strip().split('\t'))
#header.value

In [ ]:
data.take(10)

In [ ]:
S3_FILE_PATTERN.format('20160621')

In [ ]:
os.getenv('AWS_SECRET_ACCESS_KEY')
S3_FIRST_PASS_PATTERN.format(log_date='2015-12-01',site='rd')

In [ ]:
END_DATE = datetime(2016,5,9).date()
START_DATE = datetime(2016,5,5).date()
dates = list(period_delta(START_DATE, END_DATE))
dates

In [ ]:
def make_site_path(d,s):
    return S3_FIRST_PASS_PATTERN.format(log_date=d.strftime('%Y-%m-%d'),
                site=s)
def make_path(d,SITE_LIST):
    siteslist=[]
    for s in SITE_LIST:
        link = make_site_path(d,s)
        siteslist.append(link)
    return siteslist
    
input_paths = [make_path(d,SITE_LIST) for d in dates]
input_paths

In [ ]:
data_S3_FIRST_PASS_PATTERN = sqlCtx.read.parquet(S3_FIRST_PASS_PATTERN.format(log_date='2015-12-01',site='rd'))
#data_S3_FIRST_PASS_PATTERN.filter(len(data_S3_FIRST_PASS_PATTERN.sessions) > 1)
#data_S3_FIRST_PASS_PATTERNrdd = data_S3_FIRST_PASS_PATTERN.rdd
#fdata = data_S3_FIRST_PASS_PATTERNrdd.filter(lambda x : x if x[1]!="no ruid" else None).filter(lambda x: x is not None)
#data_S3_FIRST_PASS_PATTERNsamp = data_S3_FIRST_PASS_PATTERN.sample(False,0.01)
#fdata = data_S3_FIRST_PASS_PATTERNsamp.filter(lambda x : x if len(x[2])>1 else None).filter(lambda x: x is not None)
#fdata = fdata.filter(lambda x : x if len(x[2]) > 3 else None).filter(lambda x: x is not None)
#fdata.count()
#data_S3_FIRST_PASS_PATTERN.take(10)
#data_S3_FIRST_PASS_PATTERNrdd = data_S3_FIRST_PASS_PATTERN.rdd
#fdata = data_S3_FIRST_PASS_PATTERNrdd.filter(lambda x : x if len(x[2])>1 else None).filter(lambda x: x is not None)
#fdata.take(4)
data_S3_FIRST_PASS_PATTERN.take(12)

In [ ]:
def getMoreThanOne(row):
    sessions = row[2]
    for s in sessions:
        if 'conditions_selected' in s.keys():
            return row
        else: return None
data = data_S3_FIRST_PASS_PATTERN.map(getMoreThanOne).filter(lambda a: a is not None)
data.count()

In [ ]:
def get_users_multiple_visits():
    def inner_get_users_multiple_visits(row):
        sessionData=row[2]
        page_paths_all = []
        i=0
        for oneSession in sessionData:
            if 'epid' in oneSession.keys():
                epids = ast.literal_eval(oneSession['epid'])
                if(len(epids)>0):
                    return row[0]
    return inner_get_users_multiple_visits

def getVisitData(visitorFirstPassData):
    vData = visitorFirstPassData.map(get_users_multiple_visits()).filter(lambda x: x is not None)
    return vData
visitDatafromFP = getVisitData(data_S3_FIRST_PASS_PATTERN)
visitDatafromFP.count()
#visitDatafromFPFM = visitDatafromFP.flatMap(lambda xs: [x for x in xs])

In [ ]:
from urllib.parse import urlparse

def getPaths(row):
    parsed = urlparse('//'+row)
    return parsed.path

sharedPaths = visitDatafromFPFM.map(getPaths)
sharedPaths.take(10)

def getStrings(row):
    strings=[]
    slashSplit = row.split('/')
    for str in slashSplit:
        hyphenSplit = str.split('-')
        if len(hyphenSplit)>0:
            strings.extend(hyphenSplit)
    if(len(strings)>0):
        return strings
    else: return None

sharedStrings = sharedPaths.map(getStrings)
sharedStringsFM = sharedStrings.flatMap(lambda xs: [(x,1) for x in xs])
sharedStringsFM = sharedStringsFM.filter(lambda x: x[0] !='')
sharedStringsFM.reduceByKey(add).takeOrdered(62, key = lambda x: -x[1])

In [ ]:
def get_users_multiple_visits():
    def inner_get_users_multiple_visits(row):
        sessionData=row[2]
        FBref=[]
        externalReferers = []
        ns_campaign='no_ns_campaign'
        email_campaign='no_email_campaign'
        external_referrer = 'no_external_referrer'
        for oneSession in sessionData:
            if 'shared_on_pinterest' in oneSession.keys():
                FBref.append(ast.literal_eval(oneSession['shared_on_pinterest']))
        if len(FBref)>0:
            return FBref
        else: return None
    return inner_get_users_multiple_visits

def getVisitData(visitorFirstPassData):
    vData = visitorFirstPassData.map(get_users_multiple_visits()).filter(lambda x: x is not None)
    return vData
visitDatafromFP = getVisitData(data_S3_FIRST_PASS_PATTERN)
visitDatafromFP = visitDatafromFP.flatMap(lambda xs: [x for x in xs])
visitDatafromFP = visitDatafromFP.flatMap(lambda xs: [x for x in xs])
visitDatafromFP.count()
#visitDatafromFP = visitDatafromFP.filter(lambda x: x if x[1]=='no_ns_campaign' and x[2]=='no_email_campaign' else None)
#visitDatafromFP = visitDatafromFP.filter(lambda x: x if x[1]!='no ruid' else None)
#visitDatafromFP = visitDatafromFP.filter(lambda x: x if len(x[2])>1 else None)
#visitDatafromFPdf = sqlContext.createDataFrame(visitDatafromFP)
#visitDatafromFPdf.write.parquet('/home/analytics/testdata/orderrows')

In [ ]:
orderRows = sqlContext.read.parquet('/home/analytics/testdata/orderrows')
orderRows.count()

In [ ]:
def get_users_multiple_visits():
    def inner_get_users_multiple_visits(row):
        if row[1]=='no ruid':
            return row
        else: return None
    return inner_get_users_multiple_visits

def getVisitData(visitorFirstPassData):
    vData = visitorFirstPassData.map(get_users_multiple_visits()).filter(lambda x: x is not None)
    return vData
visitDatafromFP = getVisitData(data_S3_FIRST_PASS_PATTERN)
visitDatafromFP.count()

In [ ]:
def get_users_multiple_visits():
    def inner_get_users_multiple_visits(row):
        sessionData=row[2]
        pagePaths = None
        userAgentsList=[]
        pageVisits=0 # it will be zero if visits less than 1
                    #only if page visits are nmore than 1 in a session will it be increased
        for oneSession in sessionData:
            pageVisits =pageVisits+1
            userAgentsList.append(oneSession['user_agent'])
            return (row[0],pagePaths)
    return inner_get_users_multiple_visits

def getVisitData(visitorFirstPassData):
    vData = visitorFirstPassData.map(get_users_multiple_visits()).filter(lambda x: x is not None)
    return vData
visitDatafromFP = getVisitData(data_S3_FIRST_PASS_PATTERN)
visitDatafromFP.take(2)
#visitDatafromFP = visitDatafromFP.filter(lambda x: x if x[1] is not None else None).filter(lambda x: x is not None)
#visitDatafromFP.coalesce(1).saveAsTextFile("/home/analytics/testdata/searchterms/rd")

In [ ]:
f = open('/home/analytics/testdata/searchterms/toh/part-00000', 'r')
wF = open('/home/analytics/testdata/searchterms/rd/transformed_toh.csv','w')
for line in f:
    line = line.replace("[",'').replace("]","").replace(",","").replace("'","")
    wF.write(line)

In [ ]:
from user_agents import parse
#user_agent = parse(visitDatafromFP.take(1)[0][1])
#user_agent.is_mobile

def checkIfMobile(x):
    user_agent = parse(x[1])
    if (user_agent.is_mobile or user_agent.is_tablet):
        return (x[0],1)

modileUsers = visitDatafromFP.map(checkIfMobile).filter(lambda x: x is not None).reduceByKey(add)
modileUsers.count()

In [ ]:
visitDatafromFP = visitDatafromFP.filter(lambda x : x if len(x[1].split("],"))>2 else None).filter(lambda x: x is not None)
visitDatafromFP.count()

In [ ]:
S3_PageCountPattern = S3_PAGE_COUNT_PATTERN.format(log_date="2016-04-05")
S3_SiteCountPattern = S3_SITE_COUNT_PATTERN.format(log_date="2016-04-05")

S3_PageCountData= sqlContext.read.parquet(S3_PageCountPattern)
S3_SiteCountData = sqlContext.read.parquet(S3_SiteCountPattern)
S3_SiteCountData.toPandas()
S3_PageCountData.toPandas()

In [ ]:
data_cat = S3_SITE_CAT_FILE_PATTERN % ("2015-12-01","_site-category-subcat")
x = sqlCtx.read.parquet(data_cat)
x.toPandas()

In [ ]:
dates = [datetime(2016,4,x) for x in range(1,6)]
#cats = site_combos_to_process(dates)
dates

In [ ]:
sa = SessionAggregator(datetime(2016, 7, 2), 
        datetime(2016, 7, 3))
from inspect import getmembers
header = ['visitor_id', 'site']
for member in getmembers(sa):
    if callable(member[1]) and not member[0].startswith('_'):
        header.append(member[0])
header
header = sc.broadcast(header)
header.value

In [ ]:
s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-06-11",end="2016-06-12",source="parquet"))

In [ ]:
s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-07-02",end="2016-07-03",source="parquet"))
def getMappedData(header):
    def innerGetMappedData(row):
        splittedRow = row.split('\t')
        row = dict(zip(header.value[:len(splittedRow)], splittedRow))
        return row
    return innerGetMappedData

def mapData(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData(header)).filter(lambda x: x is not None)
    return vData
visitFinalData = mapData(s3_visitor_data)
visitFinalData.take(2)

In [ ]:
s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-05-22",end="2016-07-03",source="parquet"))
#s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-06-11",end="2016-06-12",source="parquet"))
def getMappedData(header):
    def innerGetMappedData(row):
        splittedRow = row.split('\t')
        row = dict(zip(header.value[:len(splittedRow)], splittedRow))
        return row
    return innerGetMappedData

def mapData(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData(header)).filter(lambda x: x is not None)
    return vData
visitFinalData = mapData(s3_visitor_data)
#visitFinalData = visitFinalData.filter(lambda x: x if x['twitter_referral']!='0' \
#                                       and x['twitter_referral']!='twitter_referral' else None)

#def getRefCount(row):
#    return float(row['twitter_referral'])
#visitFinalData = visitFinalData.map(getRefCount)
#visitFinalData.sum()
VFD=visitFinalData.filter(lambda x: x['closed_hc_popup']!='')
VFD.count()
#VFD.saveAsTextFile(path='s3n://rdmsr-anl/test_data/2016-05-22_to_2016-07-03_HC_Submitted_final_pass',\
#                         compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")
#VFDsample = VFD.sample(False,0.4)
VFD.saveAsTextFile(path='/home/analytics/testdata/2016-05-22_to_2016-07-03_HC_not_submitted_from_final_pass',\
                        compressionCodecClass="org.apache.hadoop.io.compress.GzipCodec")

In [ ]:
hc_submitted_users = sc.textFile('s3n://rdmsr-anl/test_data/2016-05-22_to_2016-07-03_HC_Submitted_final_pass')
def get_average_visit_time_per_day(row):
    row = ast.literal_eval(row)
    if row['average_visit_time_per_day']:
        dates = row['average_visit_time_per_day'].split(',')
        for date1 in dates:
            if len(date1.split(':'))>1:
                time_str = date1.split(':')[1]
                time_spent = int(time_str)/60
                if time_spent < 5:
                    return 5
                elif 5 <= time_spent < 10:
                    return 10
                elif 10 <= time_spent < 15:
                    return 15
                else:
                    return 20
hcdates = hc_submitted_users.map(get_average_visit_time_per_day).filter(lambda x: x is not None)
hcdatesList = hcdates.collect()
from collections import Counter
print(Counter(hcdatesList))

In [ ]:
hc_not_submitted_users = sc.textFile('/home/analytics/testdata/2016-05-22_to_2016-07-03_HC_not_submitted')

In [ ]:
def getContentTerms(row):
    contentTerms=None
    termTime=[]
    terms=''
    if(row['facebook_content_terms']!='' and row['facebook_content_terms']!=None):
        contentTerms=row['facebook_content_terms']
        contentTermsList = contentTerms.split(',')
        for term in contentTermsList:
            termSplit = term.split(':')
            if(len(termSplit)==3):
                terml=[]
                terml.append(termSplit[0])
                terml.append(termSplit[2])
                termTime.append(terml)
    if(len(termTime)>0):
        return termTime
    else:
        return None
visitFinalDataterms = visitFinalData.map(getContentTerms).filter(lambda x: x is not None)
#visitFinalDataterms = visitFinalDataterms.flatMap(lambda xs: [x for x in xs])
visitFinalDataterms.count()

In [ ]:
#visitFinalData.filter(lambda x : None if isinstance(int(x['number_days_since_last_visit']),int) else x)\
#                        .filter(lambda x: x is not None)
visitFinalDatafiltered = visitFinalData.filter(lambda x : x if (str(x['number_days_since_last_visit'])=='1' or\
                                                               str(x['number_days_since_last_visit'])=='2' or\
                                                               str(x['number_days_since_last_visit'])=='3')  else None)\
                            .filter(lambda x: x is not None)
#visitFinalData.filter(lambda x: x if x['first_external_referrer_domain_to_site']!='google' else None)
#visitFinalData.take(10)
#visitFinalDatafiltered.filter(lambda x: x if x[ip_address''])
visitFinalDatafiltered = visitFinalDatafiltered.filter(lambda x: x if (x['ruid']!="" and x['ruid']!="no ruid" ) else None)\
                                                    .filter(lambda x: x is not None)

In [ ]:
visitFinalDataVF = visitFinalDatafiltered.filter(lambda x : x if float(x['average_page_views_per_visit'])>float(2) else None)\
                            .filter(lambda x: x is not None)
visitFinalDataVF.count()

In [ ]:
def getMappedData():
    def innerGetMappedData(row):
        return (row['ruid'])
    return innerGetMappedData

def getVidDate(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData()).filter(lambda x: x is not None)
    return vData
visitFinalDataonlyRUIDS = getVidDate(visitFinalDataVF)
distinctRUIDs = visitFinalDataonlyRUIDS.distinct()
distinctRUIDs.coalesce(1).saveAsTextFile("/home/analytics/testdata/unique_ruids")

In [ ]:
## check if there are any special characters
import string
def checkSpecialCharacters(x):
    invalidChars = set(string.punctuation.replace("-", ""))
    if any(char in invalidChars for char in x):
        return None
    else:
        return x

distinctRUIDs = distinctRUIDs.map(checkSpecialCharacters).filter(lambda x: x is not None)
distinctRUIDs.count()

In [ ]:
def list_to_csv_str(x):
    return x

lines = distinctRUIDs.map(list_to_csv_str)
#lines.take(10)
#lines.take(10)
lines.coalesce(1).saveAsTextFile("/home/analytics/testdata/ruids_formatted_noQuotes")

In [ ]:
file = open('/home/analytics/testdata/ruids_formatted/part-00000')
text = file.read()
mytext = "".join(text.split("\n"))
ruidscsv = open('/home/analytics/testdata/unique_ruidscsv/ruids.csv', 'w')
ruidscsv.write(mytext)
ruidscsv.close()

In [ ]:
def getMappedData():
    def innerGetMappedData(row):
        return row['ip_address']
    return innerGetMappedData

def getDistinctRUIDs(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData()).filter(lambda x: x is not None)
    return vData
visitFinalDataonlyRUIDS = getDistinctRUIDs(visitFinalDatafiltered)

visitFinalDataonlyRUIDSDistinct = visitFinalDataonlyRUIDS.distinct()
visitFinalDataonlyRUIDSDistinct.take(100)

In [ ]:
userIdCountData = sqlContext.read.parquet("s3n://rdmsr-anl/daily_all_ids_by_site/2016-04-06/log_date=2016-04-06/site=rd/")
userIdCountData.take(10)

In [ ]:
def create_set(x):
    s = set()
    if x:
        s.add(x)
    return s
def combine_set(x, y):
    if not isinstance(x, set):
        x = set()
    if y:
        x.add(y)
    return x
def combine_sets(x, y):
    if not isinstance(x, set):
        x = set()
    if not isinstance(y, set):
        y = set()
    x = x.union(y)
    return x

In [ ]:
data = data.map(process_row(header, sql=False, 
                    keep=['ns_vid', '_site']), True)\
                    .filter(lambda x: x != None)
data = data.map(lambda x: (x['_site'], x['ns_vid']), True)
data = data.combineByKey(create_set, combine_set, combine_sets)
data.take(10)

In [ ]:
dataSamp = dataSamp.combineByKey(create_set, combine_set, combine_sets)
dataSamp.take(10)

In [ ]:
sampleData = data.sample(False,0.05)
rows = [process_row(header)(r) for r in sampleData.take(20)]
rows = [r for r in rows if r]

In [ ]:
row=data.take(10)[2].strip().split('\t')
header = sc.broadcast(data.first().strip().split('\t'))
header.value[:len(row)]
row = dict(zip(header.value[:len(row)], row))
row

In [ ]:
raw = apply_process_row(data, header, 15)
raw.persist()
raw = raw.filter(lambda a: a if a[0][0]==('71.195.95.193#3EBEB46E','tasteofhome') else None)
raw.take(10)
#lambda x,y: x if x==y else x+":"+y

In [3]:
raw = apply_process_row(data, header, 20)
#['link_module'] =='eBook Sign Up Popup'
#raw.take(2)

In [ ]:
def getAggregatedData(row):
    sess = row[1]
    url_set = set()
    for s in sess:
        url_set.add(s['referrer']) if 'search' in s['referrer'] \
        and 'google' in s['referrer']  and 'googlequicksearchbox' not in s['referrer'] else None
    url_list = list(url_set)
    parsed = []
    for u in url_list:
        parsed.extend(parse_qs(urlparse(u).query)['q'])
    return parsed if parsed else None
ftdat = raw.map(getAggregatedData).filter(lambda x : x is not None)
ftdat.take(20)

In [ ]:
def getAggregatedData(row):
    return ((row[1][0]['ns_vid']),(row[1][0]['ruid'], row[1][0]['epid'], ))
ftdat = raw.map(getpagepaths).filter(lambda x : x is not None)

In [ ]:
raw.take(1)

In [ ]:
def getpagepaths(row):
    return (row[1][0]['ns_vid'],row[1][0]['epid'])
ftdat = raw.map(getpagepaths).filter(lambda x : x is not None)
combined = ftdat.combineByKey(create_set, combine_set, combine_sets)

In [ ]:
combined.take(10)

In [ ]:
fg = combined.filter(lambda a: len(a[1])>2)
fg.take(10)

In [4]:
def getpagepaths(row):
    sessions = row[1]
    for s in sessions:
        if s['age_check']!='':
            return row
ftdat = raw.map(getpagepaths).filter(lambda x : x is not None)
ftdat.take(10)

[(('15231232-6606-1016-9011-714718902440', 'familyhandyman'),
  [{'_site': 'familyhandyman',
    '_sweepstakes_and_discounts_from_the_fmaily_handyman._signup_event': '',
    'adblock': '',
    'add_bookmark_event': '',
    'add_groc_event': '',
    'age_check': '',
    'agent': 'Mozilla/5.0 (Linux; Android 4.0.4; KFJWI Build/IMM76D) AppleWebKit/537.36 (KHTML, like Gecko) Silk/52.1.79 like Chrome/52.0.2743.98 Safari/537.36',
    'autocomplete_event': '',
    'autocomplete_recipe_event': '',
    'back_to_school_signup_event': '',
    'birds__blooms_signup_event': '',
    'breadcrumb': '',
    'breadcrumb_level': '',
    'breadcrumb_name': '',
    'building_and_construction_professionals_tips__hints_signup_event': '',
    'campaign': 'ppc-taboola_crunchmind-buzzbeagle',
    'cancel_mag_event': '',
    'cancel_ser_event': '',
    'category': 'skills',
    'channel_level': 'skills-3',
    'christmas_cookie_countdown_signup_event': '',
    'comments_event': '',
    'conditions_selected': '',

In [ ]:
def getpagepaths(row):
    sessions = row[1]
    all_urls = []
    for s in sessions:
        if s['age_check']!='':
        return row
ftdat = raw.map(getpagepaths).filter(lambda x : x is not None)
ftdatFM = ftdat.flatMap(lambda xs: [x for x in xs])

def getQueryStrings(url):
    q = urlparse(url).query
    if q != '':
        return parse_qs(q)
def processURL(row):
    qs = getQueryStrings(row)
    return qs
qs = ftdatFM.map(processURL).filter(lambda x: x is not None)

def getTuples(row):
    listTuples = []
    for k,v in row.items():
        if (len(v)>1):
            for val in v:
                tup = (k,val)
                listTuples.append(tup)
        else: 
            tup=(k,v)
            listTuples.append(tup)
    return listTuples
def createList(x):
    return [x]

def combineList(x,y):
    x.append(y)
    return x

def combineLists(x, y):
    x.extend(y)
    return x


#qs1 = qs.flatMap(getTuples).combineByKey(createList, combineList, combineLists, 5)
#qs1.count()
qs1 = qs.flatMap(getTuples)

In [ ]:
qs2 = qs1.groupByKey()

In [ ]:
qs3 = qs2.map(lambda x : (x[0], list(x[1])))
qs3.saveAsTextFile('s3n://rdmsr-anl/query_strings/2016-05-29/20maps')


In [ ]:
#qs3 = qs2.map(lambda x : (x[0], list(x[1])))
#qs3.saveAsTextFile('/home/analytics/testdata/query_strings/')
qs_local = sc.textFile('s3n://rdmsr-anl/query_strings/2016-05-29/20maps')
qs_localsamp = qs_local.sample(False,0.5)
def getallqstr(row):
    row1 = ast.literal_eval(row)
    #vals = list(set(row1[1]))
    #return (row1[0],vals)
    vals = [item for sublist in row1[1] for item in sublist]
    vals_set = {i:vals.count(i) for i in vals}
    return (row1[0],vals_set)
qs4 = qs_localsamp.map(getallqstr).filter(lambda x : x is not None)
qs4.saveAsTextFile('s3n://rdmsr-anl/test_data/query_strings_with_counts7_1/2016-06-02/')
#qs5 = qs4.flatMap(lambda xs: [x for x in xs])
#qs5 = qs5.flatMap(lambda xs: [x for x in xs])
#qs5.distinct().collect()

In [ ]:
qs_2 = sc.textFile('s3n://rdmsr-anl/test_data/query_strings_with_counts2/2016-05-29/')


def convertToDict(row):
    result_array = []
    row1 = ast.literal_eval(row)
    for k,v in row1[1].items():
        string = '"'+row1[0]+'","'+str(k)+'","'+str(v)+'"'
        result_array.append(string)
    return result_array
b= qs_2.flatMap(convertToDict)

b.coalesce(1).saveAsTextFile("/home/analytics/testdata/query_strings/")

In [ ]:
import operator
countedQS = sc.textFile('s3n://rdmsr-anl/test_data/query_strings_with_counts2/2016-05-29/')

def getOrdered(row):
    row1 = ast.literal_eval(row)
    sorted_x = sorted(row1[1].items(), key=operator.itemgetter(1),reverse=True)
    return (row1[0],sorted_x)
    
a = countedQS.map(getOrdered)

#def getMostRepeated(row):
#    return (row[0],row[1][0])
#b=a.map(getMostRepeated)
a.collect()

In [ ]:
def getConditions_selectedCount(row):
    sessions = row[1]
    for s in sessions:
        if s['link_name']=='I want my free eBook' and s['link_module'] =='eBook Sign Up Popup':
            return (s['epid'])
ftdat = raw.map(getConditions_selectedCount).filter(lambda x : x is not None)
epids_l = ftdat.collect()

In [ ]:
len(epids_l)

In [ ]:
existingones = ['9a87816b7b1551b0da937bbcb2879e57982690d1',
'743d58872b908258d9b54f9ab9ac064eea96c5e1',
'52b9ff84baf3dcece350a4fbe9b044df6864648f',
'988F899E97EB848603A8A762FC8C3756AB6AB958',
'c5b995fba35d8679ccec98d35b545dcb39cfd880',
'51a6f0e4d5b38f68fd0781efa6f991b2d782e51d',
'dd6b53dfd7c18b9faaf6ca4bafd2fced025c6a0',
'3931bbbc300e67c5efd5dceda5e4fee271a9a87',
'bd8956fe2fd9644468a59fd1c17108a8042976fe',
'f8ed748f369da8ea28c96aedbe3b1e26afef8d0b',
'c6e6c0eace6613906c2a166ba0932c1a62a01621',
'6907dadc53eac676599108cf9fd8aa10e273f96a',
'6b81e674d2701aedb615fb91342ac4d08cd77757',
'94c348ce169b0dcb2ac330a28b883ee24bfee79d',
'0C225BABD609492465590C66BEEDD5E00D919CDC',
'F14FDAD4423E5CD3A338BEA1590FE56819516160',
'93C284D4951A93977E31E03A488C34CB087A43A3',
'5590F04E69F590A77790C7F3B6A590449BBD0F0C',
'a87d103c5365129feed83ac0c344cb4f2ba15c2f',
'1c3a42f9afa10be770d20887ef9c8cdad663303a',
'438f039d5d922ee8b425cef439304940149ec858',
'e5bd94258637c384a49585c5f9c54403fa0ee518',
'5918754c53ff10effb5db9ac2aea0c3fb61f5f18',
'f1c70eb5e369449d333c8c7fd8a5583bbfc0cd89',
'b9667c9f77976d6434fb5275f7a3f5fe1fcbcce2',
'4bb782eed6b7e09441d844c76553c50701729d73',
'd0ef3d3d639651c52c1af0d537c7c4844536c8af',
'5d46932aff7982f256f4452bb9c0b43c8db71342',
'BB164283FB6602521C6EF13F74C11719F4C7088F',
'5CE70D00D9805DDB1B0D488727351C0401FBD274',
'819143D7D91510BB91E529A96D21D86A411EC139',
'4AC3D21F9862E66DCE52006031216ACC3208AEC5',
'C071352E916CFAA0D49B2CB77F09867B4A46FAE0',
'5244ed16524a9e6dfe75b66a72cbbda793d50814',
'49df04a58e70cf797fdc2905f10d40254ba22570',
'6c1ca2a6b9cd7c29b019b40926c4e91a932a094a',
'4179a1832f3a895fcc8b7be4409568428f0e55e9',
'50CB32E4117124F82765348C498EAC427C1AE525',
'36F288251C760CB2DF9D0974D901B99D1C6CDE37',
'83dfa8d878c20bb2b8c5440e1db03e4e3e3ba52f',
'c0c65b2b9f146676ec61b7003a2b9a68bdf34dfe',
'218c3f0c838d752b350eeaab5a09f49e24fb23bb',
'575a895e00f6ec6270bab64180409a1c9f702d5f',
'cb47cecb3eda91aa62a2422add3d6e6df386c805',
'aadc03650318e07143b78d4ae9d9fa23ed732dfb',
'f05434c0b4d8a7960eb9a5fe0f1a5d39ebd07ac2',
'95b437e9bddef22d0f9cef605449d2d4d378ce22',
'b5e6fd15cba6311a7e5ccef97da3a9bd09143cad',
'538FDD60AB9960B0CFD02254DF7BA14FF983F4C0',
'b04fc475ae163082e3858ca22bf973248494bcfc',
'b76bda1f9544bef7432dd7802aac37668a344d6a',
'83291e629fb38036b32ad663c6fac7586cef3f7c',
'8F1BD367567BCB3DE15D155E7D931B421907ED7F',
'BFBE2D446F48F5FAC277C2C6446A269125D1A701',
'034662339E6BBB0CD52A8E604C010A082F1DAA9D',
'514a5d1cccfcde3806d2aacad09fdf5e93eb01d0',
'a61fbef8072edb09e62cf935ab9fd92d4b6fb2b7',
'35fceae7013ab16100966ce692d679651949b97b',
'73aeb954541ea8b7a6f92b4b1f839c8f369ce216',
'afc250797b1cb9ba463eed965164acd24ef226cb',
'2db8b74eaacd9e151804b409c8aef1b0a5f8dc58',
'faaae12eb8195e96c00fe8604dc5bc785e3c6e5d',
'21a0d08fa7c5840063779a2796a33db660400385',
'1067f89308cdc07eb0c76f21a747d6842913bf4d',
'6c724d035af938848fd078cac5f97c916e63ac8d',
'f1e06d92f3c9f21cc5af7dffdad3d580cc70df4',
'da780deaa4c37cf51b05bfc03620820ca3b18223',
'28e29689810b0418f56b911d154d0960551b56d',
'79ae4626b663edd1a64ddb3fbdcc0ead32bacc7d',
'4f2177ed51fe2cfc69d674767bc88c41c21eed6f',
'4c67b920e915997904ee01ebcbfc9f09294d93f2',
'216f21fa0c151da7f39d6146679cdf405d73eb57',
'6bcc87eae35b41da161eb913e7d065afd0cd77bc',
'dd51b7dc50d93fb1584d98d4d18dcac05081c05',
'265c2586954ca5f24600c5ed0c59ed0093456f3a',
'dd5477df791871065d2bf11128d9d039fc4d1d03',
'65951b8cee49782d4efebd8edbe764fcc6bb4938',
'ab878555e7cde6ef0316a3e41b7a89b51ae812b0',
'138F3214DD29D65E07685DA40C546648C4D93336',
'636A41343D4D5D9A5FBF0F9BF128119E07D7C399',
'80E65B3571A96733A8D3CC0F1FD1AF3681A20352',
'1E3A81AB59B7E778C6A5ABCE0C20C0DFC7E3BECE',
'F9CC37367603D91B5AA4BB34EF9D36BFEFC87B54',
'A98EE6DE7931CA6C2DC28F28A340F00D34A963D6',
'03F9E1DBAB8BCFBCC27B8E8D1208F71E926E36DF',
'C0D000B89D5582E4B0856D80600E7B03FB4EF93E',
'AF1F75ABF00C64D6E584C46FBF6F7690FB56B9B0',
'feb2ce95db0792f4a8aa35a631556fcbeda27189',
'939ab67ced96e8a7c679ef39ea4226e09e2c28b5',
'7d787786fe109ecb257be60ad01641f8ec23ff25',
'b7c77df98b16527492e8fa140c2be47d2d78ae7e',
'85FFFD44E1D17B27BD3B920E9147A3705D6ABBFD',
'f7ac1818ba317c8a70e734d7d1d86d61f28766e4',
'b8d61da84690f4e3567bef0a60c1acb7a86ae7a1',
'e4bd7f09c89394ceedba36c35594239b27bd1b72',
'd18cd78606dfcaa6377742f5d91f1ba5e3e033db',
'ca69f16be7e74fe9031ef9e613ab7092ac5d27ed',
'e3d62e2b74acdcf5b51966682b25fb71b0794006',
'5f6821874f8c9c081659a53c090f049c16ef47ca',
'2b5aaac8fa9b0263966edf4423733736b8bbfc29',
'f3b12385c4d37f155ddef2a0c63b1f3233dd05b',
'1fdcba68f659cadae9ba891686fd8531e46ec90b',
'2d84303644105d8863a78fa54a752a4bb9d75884',
'67d5d08d26edd5501d9632ad05fce2cb0d56ed82',
'c81a83d9131715992b134b5fc620cc5481b29b56',
'757ba3195c8cda0814d86102a52118cda62acfe2',
'f88a3ca4a8d98fd413b44ffe25cad83e7ff31379',
'1f999d79eb491bfa3ceeccf6279fa8f35220e638',
'dc9f66eaa70b5aeb903f9d211c5b158961238344',
'3992e3900b8cecce3aa96efeeead533db34b5502',
'a198e4bb7af3a7f8a1c75334e4688ddf758a10bc',
'da1124c8a7ff021e2ae69aa866033e3fb5f2d578',
'83cdab4e8dedd04a0d5f3784009637c674ef1e2e',
'ff298f7b946eab1b366545f61f10c1e5602506cb',
'b94a0dfa869a197c5bc41989a3ec5939e97692d6',
'f0250cfc60b24d0b0b3da7d27551883e784fd65f',
'904D57905787EF3E4CC67C652DFA777DB2B63F80',
'214223886DF0962E59603F0FAD7AAB301B32204A',
'1dff12080d2fc60fc63816bd86c6d8a38bafb28e',
'7212f9d414a92c35bf286f738ab315d79e322c3',
'dd2f9f33fd5c24d8e7b2e8bbe1f3d93a6e89b982',
'bf32640e2f58074d1de2fa5fe2898bbd5fd0982',
'1138be18daa8ef13ac4f5f686830d4b6d25b5d16',
'C39E17EBCC9A11D79E8BC8DC0868D1B34AE8D1A8',
'D1050AC93A8121B047BE23FB9E0809A3A292CF7A',
'2368D02A07111A49CAEB796946D75D020C6777BB',
'0E35C8ED4CA5B2A600BE503E8773F4A3ABBF6E41',
'f207081501461276994a8c588a868352e69cfed0',
'7ddface7acc13548b70a6ea84abcee53b66f9538',
'd70a0ce78491c18698ad1e5a3091db5393c55426',
'c9b26031e9a822cda6443ccb043b6fb9cf17920e',
'A81990E41AEC54CC84EE8F31BD519C0233691471',
'F5F8A2DCC5522F3E3F94EF11AD108444ED4DF822',
'b23492a6ee9d6c383bf327525b335f420f210f7c',
'8542f5f27ba91641e882266c43a8490c57998e7a',
'946C38EDAAC4279BC9F67614490183C4A5852B8A',
'CD71637EF1FCE05F3CC5F61802B8E4E519F73883',
'17BC80F0D66F24B7D28E95665424277EEFCA9CA1',
'd545178fa24192a2c465f499eaef4eda5c370762',
'af5d7db5edcf977015bcc9954020ba1f817bb163',
'6775476c21e7a37757b96cf16bccdfd82359cc00',
'289b6fe9cb3acacccd670846460234b3029e2eb8',
'93f2c544e63d793605e776d27e6f3abe4be6047a',
'8bc4c25c3f62616e5f1286a6d350ed35206903a8',
'4d1c8d90b4ad63a6634ee4388f53104012827b17',
'634F783CCDB13006CC26A0BBCA4BDA85220EB03F',
'653d13f61a46da356d94078d6b116c3b9474eddb',
'5F761C8271A3D668F3D84D4519F8E6CED29C1837',
'20fa3823f21656da31c952e4544b436d5414f52c',
'af3d476b03e02c5f56bb079cf6e5ffd5362122ee',
'3f869b737538c6a512243d0ffd09bdd823b433',
'80b1b91ed84fc84ee1df8cf6a07e2c58aca548dd',
'ceae3ab700553d0e2b05962c450c5529fb650529',
'3f080d1d556fffa42683b2cc3de6c03879ad3b3c',
'9ec118d5a131f2b459a99c63a94ac11b9cb8241c',
'35785dba25df9a5e97b26a7cf123abdf0288777',
'23ae2d40e6107b9ad264cc7931141494bfa58195',
'a1d2c9ebdecc8af8f7cba730d446de2bd20a2f9c',
'66e29ba47eb29ec075517b92bd7b2cc8d8a4c026',
'C1D9E085752EBFBFF3B69BF88B993DC485D47C30',
'02B9C4F8B2463DDD1F8C95FED4065C6914099B8A',
'aaf3425037b00c8cfd2e593abfb57319b09f7a8b',
'ef32260e8e954f9f676123cb440945c772100021',
'ac5d218d933f41cd59179bb76c0684f076a69853',
'180388f496300253662e4b311fc7e609e345c6c5',
'6af5733c00a1e91de87fd5512a000874282abb5d',
'6a0d49e9cdea00aa23a39d7148b9a4e2f4df19c9',
'84dd5fc3254233e4f18aa1b536403a875b9c8f5b',
'2b92af87861a1f554e0cc16d4f063114cf832f51',
'd44c747ea192282b7b98ea32e3016b5ab7c6b693',
'b5242258f99bfe80815bd04ef14169b1220c443b',
'7364c1332060a9998b6e6dad7917acf3280fe3c2',
'9f458819f230c57c4419b5b2f489687393dbd9a1',
'398a0e013001fafb3fef7847faeb550b576361f2',
'1341eeb062dfbf1197ba28d816d80b402946764e',
'7a1d73375716c6e3d62be9e927816162774317b3',
'6c83838593da34563fd8d99f405971e47cf3174b',
'ff6567dcd9f6e69b283acb8bb6e991f5a0d5d8d4',
'15b306aa90e81e1e4d4dc4e804dae619f4206bbe',
'5af2b4ea72a474a0f738de2344b98551d7b7fc03',
'744316b665c4c2f189c16221102207d9ada5f0ed',
'2d50c4696f7c037a8be79838b3e56040ce1fbb81',
'3779920e867ebaf0ca9da4e4bfc117fe9508f965',
'cba2d39bc3338a653f0e3a236b9e9f250155dd59',
'5d6e6c7c7214fd6d173d7101b490f435a0d8ba83',
'dccc256bca71c6566fb0d1306d04a0ec10028f02',
'96CC45A0387657C2611D1FA9A12D80AC14A4FD97',
'5A627903745DB1CEDFA86D9320CB33340AA4B3F5',
'5ba956406b4e95b469a19a39daa116d607300edc',
'51bfd29940cff55707594c14118782a1878484c2',
'144ab6e1d8ffeb813bd9be0758dd3754c53c72a4',
'7bae5a807daf06bab676630df6e160ac7a13e634',
'7028dfad01a4a19bbd78261134181d3f476991f4',
'876bc37d1b2b515f24ec0a73b4f547a7036e8a4d',
'd1ba2e2c8c912b9841ad68a8d18554a37eafb49c',
'f54770d687d065eb22c4c15457c923e891654038',
'8f6ec76b46e9c3195f4e7e4b5f2e5707b9bcf490',
'29f3619d565ffb3c04470925a0093151a913530',
'23fd6b5d0c15a832381174b1d72cebce4436fe78',
'2421C728E1F1411D9060AA9F66FEDF2FF2A83C4C',
'431252D63C4BD07E2F61C28C5A4AF04D40C0C100',
'EE149D8A8D804582C076F65130BE101E6DBB800B',
'FDEFAA4808808EF2E534284556A99A50EBA649D9',
'A5A0C9515B8EB2EDFA1B8B633AF70748C268A728',
'9B88E9F52523D4146B38CC147EB3BDD395A410A6',
'9657d32401a199dd7d4ee24b1db32dcf59e8c8af',
'b6c3bdc0a60caa14691a84e8dde7102df95490e9',
'60df18333487c63b88af2e4e077df67edd1d9dac',
'19a4cdc38272ca34bb5d4fe7d5953cb8a6c3411c',
'96e6fe96ef018361a030b75934403d7bc9b17d56',
'866080f1492f340938f1d4949284034bcbd72e45',
'5c23cce01830f7deecf677db2dd4dd6373ceddd1',
'6ad7bbf4bcac22a0e5cd7e8e5c9f8302e5dadb',
'f570f8f4d881e728be45c9b56958d598ba2408a1',
'ea0ae925aab7385dcc9a960fca673c9611abe991',
'4d262a02e78c061f3877bbfee12874fca9f6ff73',
'707f63f367aed56c8fbe1bce09c00c9aa14e6cd8',
'2f3872afc986de9522cca13c2fd0d51ec4de24e1',
'2cf0b5ec8be3f802426c4e35edd20dc7d3cf149e',
'4b650dcc9217c8536b34f9fb0a4c7e1a5cab9684',
'746f2ed17bb6f9de6a1511f3821dcfeef4526387',
'c5eb6ac1753b9d681e4aaf1a2bbcce598a7f8612',
'cd5bae4bfa9dfc2f0043c6db9a1fc7215cc09e56',
'd046c0bfb81bbea90292b8382c9f917116aa9813',
'670c1d33ecf79a82a67a77bbb72c658cbe177c70',
'5ca63750cb094b68b74c4710589e945212783747',
'89f01e3439e7275fe3a537a65a9477596871ed4',
'56fa5d791734cc7944b207c766c8d215a6b66929',
'86ed1ee69144040b27ef7db9868d37d916466533',
'ab1cf2d9ae82187f748b07d8db14f36b67f1b64',
'a297ed85ac3da9f4660cd71f477ac01e0f20ff92',
'36f2007cbf02e9183caea9ecfa1d5b6d473db836',
'da47ec0ec7ef9ccadbda0a1640d122bfeb9619db',
'5154811b2c1b8f404dd413a133434fe538491de8',
'2ae5a876ab6d1ea52fa5209995842fe775ea3f76',
'c1952a08c7bc5cc3f926c9b50a4bd3d787e18eac',
'4f4e807bd98aac32478414996f04a70c725809fc',
'a295ec2a315062a3414ff7da27cb50b274f5fd16',
'ea17a25ade63682860576a180b57badec189a574',
'd458778f00579a1231ea8dc4567626571ecccee8',
'a21e71301adb26d91aeea8ae0231b5c8dbf7c094',
'8a3bc537682b210a6f6787dec03622796ce0fc6a',
'9f55b70b9f56b7c44a6ce1b3f04e489d3e004cd5',
'46bff532ff7a4fef5b34ce10a1f8b667ba3907fc',
'2a1eaef0633358048870973b0bb65855c64552c9',
'939f2404421acebd175423b8e455d7f94b8bc03',
'6a954df8ec47ee55b4a872509e7214af3705333a',
'3f0a343c57218edfb2c3240baea68008b8c3a5b6',
'21cd06b3821d4792c7668bf088f191f892400052',
'50e49c82a0c0bab7600e6642a6099904acbb45e8',
'2da77c8b2c28ff2244dc282564578d5cdd6b159e',
'59fd2dffa6326ae0461f8e25419f47101813dc12',
'2d86683938374f530f844b9d3e1c451114ecc43',
'9ca2c008306b3e7693ad9b29cfe9feda0c7d2bb',
'f4af34d16eade1803f5ae2fce28d5e70a99bda8a',
'df7a141b057f83ef08b0f4d5b9df5d3f1a9f56',
'd99aabd12e56fa1f9e27ec7f062ca3d91a091b2f',
'69e99b0a8caadd9a65dc5a05eaf4d8e73f7d211',
'dd94a0de0560a0c08f8dbd5a3acec97238f0fc07',
'8e7b2901e616a81a91b7035489b998834379e0ba',
'4ea76121f6f3c4ba315d1e6618bf311611e957e3',
'66dea3f25187b615216b2602db78644f001bb521',
'28e14d9737aebfbdb7b0bb3854b0b795b8bf7e8c',
'1615aacab44ba608433719dd41c160f82d12e254',
'34ef05f5ca731e762460e30304b2a4fe83b4fb87',
'3d57a8935d7511c63577518fdaf827b59b26de00',
'482fcc5fdf8bbadf6b794eabeb4521de308ac15e',
'6dc42ec53cc31fde5a258827c6a9f66b1deb74a1',
'3f9d5b5ef66390971e0796146a98f31ff482d5df',
'f0c09b66a65b9390ed08bcd07d8205d873f7e091',
'e14524a2333da2ee3b6d6e12baea14cb13d6716b',
'1df5a6741d131f9165637d17a4e5e38285a6498',
'a336736930f2040fcc682f8c10b3cb7aa4b62478',
'7e683ad792c7ffdf5979c861ccb8a288d213340e',
'31fc65f8b0ff6aeb48beb2b1bafe0a7de5305535',
'4785033ec9bce48c555f6d1cdc20926248980dc5',
'8be757cd7b49f1f773b5bea266bc997b2ac76cfd',
'6b291947ff00f2d6665fe762e718327e646d8bfb',
'740d01f5445c6694f46b250a73b6c9cafd55863',
'165BE3324F5171CF9C1E62C505E5BE690F152ABF',
'75EC47286E862B139A228A39675FF488720AB8A0',
'21ae6246bab8c0d8f8b706704668ecc0746787f5',
'ff47e1ec1ca4dc5fd96c9be7875ffdebc99a696a',
'b11b9c9f97394c85059343257684b1600932afab',
'b8121f20c2f6fc942d70f667755214e103e90a2b',
'9f4c2907e06c415498dd3f74b2a6184a1276f0a6',
'6f3c552d08dc98d3d06f70c4e34a56047170ee43',
'2b712b59477e0260030a3aabfe2dcb4712fc03fc',
'6a0dca507271bad069195c3d557e75710dac8264',
'289475e2a7463147ad1670c0b30cecaa26da8772',
'39c2795581b526ecbabfbe0ab29777b8352a2cab',
'fd8a2979203ae2d763ca36c9a2eb3224c8b63e2c',
'dc273d332ef7d188ca1fb5ebd3769481f744fd33',
'3610bf63359df377f955f4db7cd638c668e52f6c',
'853452f8bcf3133ce0926912904d1963cb9a6c23',
'fed65374305df47b2a900be6a40d80b3d79119a5',
'314f657f09bd9cfc040616aad5ea7ce50779d7fb',
'f6babadf04597e7fb6b6c8931d61de587e21b9f6',
'8092dfbd5e3e807df8f71fbda84a6dc4bab471cf',
'1DDD67257594056B4A0BBC8777D4AF0FF08CCB8A',
'a271f5342b3b773673bf1e7d37200a080385a984',
'ed8f81f8153a7d09d6603e3c086aa2b204da0e33',
'54963d1a9efec31ced4c604af0f8845a1bae2621',
'6de578477b7a6e3984e1cf33fe6209ae136b35bd',
'b0935316cd44e9f3eff1df5b91db3d30ac36e9b3',
'281308926cb879ea2988e57fd44464f84ab160b',
'9d8aae63d500db4988931abe617c3bbcc74bbff1',
'881e62d6a2aab335b1cceea63bba5941cba8627c',
'63e3ce8e861b7d823fa56be5bcbf67ca6d062b97',
'f777634b2da6c62cdf797e508fd63ad1d00a74d7',
'437939084506dc7e0655f5248529d46236d032e4',
'955f264fd2df5df10253195668626a7afb9bd03b',
'3ba14037294f9ba2de679636391ac46c95472228',
'8185bf0f6fbad63ad23c6e3d2cf2ff2e86c83980',
'c24fde294c536aa978b53ec0157cd547c30f1de0',
'296f7d7fdbfbc90d9b0c9d7052e1a4a554318bca',
'775945f9dacde240d67261e80f7a4609044ac1c1',
'cc86ad5d97e5fc68a8c40159232fd0ad8d148f6',
'e5f14c5bd9b8323d689f4774a72975021021846c',
'24d937ce1fe08e450b1471f7c69d39c5ae08788b',
'23f93066d6b5abe8df0747c849960a2f942b39ea',
'7054cc8de208d17c3b732d6bb5f3340949804f06',
'198543897b9ae2346822aa900eeb271ff3f31688',
'9796675438901d2c8b63e474bd6fff161b11fb51',
'eb5f5e06545c3ee39dd5249c5690efea6c14a740',
'a58ee18d168a847d7c043960941426453fb62791',
'4bce2c90b9bce93b3fe54cf3f3890e3d8257fb1c',
'd0ff84713942c1ed04e5501a7a89c723040de459',
'9349778ea269c3d7c66e0bc23051fdcdda1c28c1',
'96fda591de40daee8bbb1f94638801884a61592b',
'1f58a7456a9342d96bc7b2ea2295c0e59f554389',
'196f9fe952a7230f8ac5c251ee212318a7c6f1a6',
'668d947453399ff187d3dc35e308eb0722d37c1b',
'f098a04beba35ef4ce7da406e77e1c1e521d9a80',
'3402e77c4fff69c0429255b98737d5dde3bba7e6',
'7cc0f1b13ae37600adccba72c537636e9f668b05',
'53f3bd6c1f6da3de3100e95c8ab2e2b174827446',
'1b572dec5a15a7dd90f313ac123409c391086324',
'cee012c592365582598d46568b00964b448f9808',
'e1ad3a847dc994e784f985785e8c72a1ea3d87bd',
'fdc7e8d2b21776f3c286e22b5549a424eb2ee7df',
'f2fd6671e66624ec139151a4db852bbade34e531',
'994192c7fc1129471e8a08112790f11c5b8671a0',
'76616947c9ec74febeb35eeb7bf0b08b50c4aff3',
'f2ed791c7974f6fd21f4734f18f05740a76814e5',
'2d31a2f1ff2c6d7a190d0cc76bcc1adf50ca996',
'3e6c517d5dab0556712832720ed4d8d198de5ee',
'6b9e18d70dda6f48a1dfcc35a1df1e3467e24164',
'acab05b1162cb9c569afbf346406698f52c37d65',
'e92bc073d212b7af54ca69cd67efb4f703696843',
'ceab83c06a498e98835104b456d5b7d0e0dc1183',
'3f95de59233ed88c4cf2bceafd8fd9035bdded2',
'b4d636a6dce6282944c70b4f7dcda65c5fce598f',
'1fdc3952e1affcb9f8fcee7f772e13f4a149304b',
'b656aad59b39bf6f5a7e594fb9da3328aa7d104a',
'27a2687481c9b9acaa327df11d58d268fc1dff96',
'adbeca137e770a9e5ffeda093f69a7e5b5d7d1c6',
'952350d82f345028fdb7326788bb41f574fc44a8',
'ad6b6c4e093cef38417900181b2cc8b630b229ca',
'cbe58f65c3423dcc13b826f3dffdae95131900e8',
'9b3a9dad79eac59699bd76c8fb97af7ba90bdab6',
'387c7c263aed9c961c6dafd6ff27c5f5527c4f47',
'f8b00d38d339f73aa634e68cee28f3a1ca75252b',
'8eb9104522f8fa5ceae1c2c5433a682f22ea388a',
'9259c306bf376285a8a94434b678c021fc49bcd7',
'3384ae17dfe12ded104a18a488ecf4d93a5091e1',
'1594004dcf3edf693f1c3e682b3192629d0457a7',
'859af187e1509555995bdaf75437cc7ff2a216ea',
'983AEF64F009C910B496A88BA1CB1BE1178A6B42',
'b5b0bb5eb2610ecbdce579a074f72723171528e5',
'358f85b304d3cda237cbc4dea7f355527b6b16ce',
'6ae633b0827b9f01c80c00d4654fd13ae660e27d',
'd979c15ea0ffb4ad68464fc7a30dbd5ba5100aa4',
'e558d5451b4da81e8ef19bdd1e59fcfd025aa39a',
'2fc6645b0da0a4b5e1b6244597998eb2929fcee3',
'd894c8ae4924cc1502fde0f600c8f8cb3e39f977',
'34c13766c304427bf7b6397159f310b484709ee7',
'4983f5bc8ac513fbaf32d6f44e7a5fb587335ddc',
'624597d7726c19647d90bc5459822a2898bc16f3',]
def diff(first, second):
    second = set(second)
    return [item for item in first if item in second]
diff(existingones,epids_l)

In [ ]:
filtered = raw.filter(lambda a : a[1][0]['link_name'] =='I want my free eBook')
filtered.count()

In [ ]:
len(epids_l)

In [ ]:
def getNsvidCountEpid(row):
    return (row[1][0]['ns_vid'],1)
nsvids = filtered.map(getNsvidCountEpid)
nsvidsred = nsvids.reduceByKey(add)
nsvidsred.count()

In [ ]:
from user_agents import parse

def getUserAgents(row):
    user_agent = parse(row[1][0]['agent'])
    return (user_agent.browser.family,1)
useragents = raw.map(getUserAgents)
useragents.persist()

In [ ]:
useragents.reduceByKey(add).takeOrdered(58, key = lambda x: -x[1])

In [ ]:
a = raw.filter(lambda x : x if (x[1][0]['ns_ts'] !='') else None)
#aDF = sqlContext.createDataFrame(a)
#aDF.write.parquet('/home/analytics/testdata/HealthCareUsers')
a.take(10)

In [ ]:
healthCareUsers = sqlContext.read.parquet('/home/analytics/testdata/HealthCareUsers')
healthCareUsers.take(1)

In [ ]:
dt = datetime(2015,4,4)
up = UserProcessor(dt)
#up.categories_viewed()
groupedValues = raw.mapValues(up)
groupedValues = groupedValues.map(lambda x: x[1], True)
groupedValues.take(4)

In [ ]:
pw = ParquetWriter(dt, sc, sqlCtx, True)
pw(groupedValues)

In [ ]:
pd = sqlCtx.read.parquet('/tmp/tmpok_3oy5ytest-parquet-write-visitor-2015-04-04/log_date=2015-04-04/')
pd.take(10)

In [ ]:
def createList(x):
    return [x]

def combineList(x,y):
    x.append(y)
    return x

def combineLists(x, y):
    x.extend(y)
    return x

def process_row_for_RUID(header):
    def inner_process_row_for_RUID(row):
        stripped = row.strip().split('\t')
        if set(stripped) == set(header.value[:len(stripped)]):
            return None
        row = row.split('\t')
        row = dict(zip(header.value[:len(row)], row))
        if row['ruid'] is '' or row['ruid'] is None:
            if row['_site'] is not '' and row['_site'] is not None:
                tup = (row['_site']+'---N',1)
                return tup
        else:
            if row['_site'] is not '' and row['_site'] is not None:
                tup = (row['_site']+'---Y',1)
                return tup
    return inner_process_row_for_RUID
def getRUID(data, header, partitions = None):
    ruidList=data.map(process_row_for_RUID(header),5).filter(lambda x: x is not None).\
    reduceByKey(add).sortByKey()
    #combineByKey(createList, combineList, combineLists, 5)
    return ruidList
ruids=getRUID(data, header,10)
for x in ruids.collect():
    print (x)


In [ ]:
# sends mail if log date data is not available.
# if available returns data and header
data,header = get_log_date('20160406',sc)

In [ ]:
write_user_ids_by_site(datetime(2016,4,6), sc, sqlContext,False, data, header)

In [ ]:
process_category_combos(data, header, cats, sqlCtx)

In [ ]:
# writes log date to dyname saying the dates are processed
conn = DynamoDBConnection()
record_log_date("2016-04-06","site_category_counts", conn = conn)

In [ ]:
def createList(x):
    return [x]

def combineList(x,y):
    x.append(y)
    return x

def combineLists(x, y):
    x.extend(y)
    return x

In [ ]:
def process_row_for_RUID(header):
    def inner_process_row_for_RUID(row):
        stripped = row.strip().split('\t')
        if set(stripped) == set(header.value[:len(stripped)]):
            return None
        row = row.split('\t')
        row = dict(zip(header.value[:len(row)], row))
        #tup = {row['ns_vid']:row['ruid']}
        if(row['_site']=='rd'):
            return (row['ns_vid'],row['ruid'])
    return inner_process_row_for_RUID
def getRUID(data, header, partitions = None):
    ruidList=data.map(process_row_for_RUID(header),5).filter(lambda x: x is not None).\
                reduceByKey(lambda x,y: x if y in x.split(":") else x+":"+y)
    #combineByKey(createList, combineList, combineLists, 5)
    return ruidList
ruids=getRUID(data, header,10)
ruids.count()

In [ ]:
ruids = ruids.filter(lambda x: x if len(x[1].split(':'))>1 else None)
ruids.count()

In [ ]:
a = process_row(header, key=['ns_vid','_site'] , sql=False, keep=None, compact = False)

In [ ]:
b=data.map(a)
b.count()

In [ ]:
def process_row123(header, key = None, keep=None, compact = False):
    def inner_process_row123(row):
        stripped = row.strip().split('\t')
        if set(stripped) == set(header.value[:len(stripped)]):
            return None
        row = row.split('\t')
        if (row[0], row[1]) == ('ns_vid', 'ns_utc'):
            return None
        row = dict(zip(header.value[:len(row)], row))
        if row['ruid'] != '':
            if row['_site'] == 'tasteofhome':
                if row['ns_vid']=='9c5c8f37-7067-0970-1513-856476450000':
                    row = {k:v for k,v in row.items() if v != ''}
                    return (row)
    return inner_process_row123
def getRUID(data, header, partitions = None):
    ruidList = data.map(process_row123(header,
                               key =['ns_vid']), partitions).\
                filter(lambda x: x is not None)
    return ruidList
ruids = getRUID(data, header,10)
ruids.take(10)

In [ ]:
def process_row123(header, key = None, keep=None, compact = False):
    def inner_process_row123(row):
        stripped = row.strip().split('\t')
        if set(stripped) == set(header.value[:len(stripped)]):
            return None
        row = row.split('\t')
        if (row[0], row[1]) == ('ns_vid', 'ns_utc'):
            return None
        row = dict(zip(header.value[:len(row)], row))
        if key:
            keys = tuple(row.get(k, None) for k in key)
            keys = keys[0] if len(keys) == 1 else keys
        if row['_site'] == 'tasteofhome':
            if row['ns_vid'] == '99.174.235.0#43C262C5':
                row = {k:v for k,v in row.items() if v != ''}
                return (keys,row)
    return inner_process_row123
def getRUID(data, header, partitions = None):
    ruidList = data.map(process_row123(header,
                               key =['ns_vid']), partitions).filter(lambda x: x is not None)
    return ruidList
ruids = getRUID(data, header,10)
ruids.take(10)

In [ ]:
ruids = ruids.filter(lambda x: None if len(x[1].split(":"))>1 else x[0])
ruids.
#ruids = ruids.distinct()
#ruids.count()

In [ ]:
def process_row123(header, key = None, keep=None, compact = False):
    def inner_process_row123(row):
        stripped = row.strip().split('\t')
        if set(stripped) == set(header.value[:len(stripped)]):
            return None
        row = row.split('\t')
        if (row[0], row[1]) == ('ns_vid', 'ns_utc'):
            return None
        row = dict(zip(header.value[:len(row)], row))
        if key:
            keys = tuple(row.get(k, None) for k in key)
            keys = keys[0] if len(keys) == 1 else keys
        if row['_site'] == 'tasteofhome':
            #if row['ruid'] != '': ############Remove for all processing
            return (keys,row['ruid'])
    return inner_process_row123
def getRUID(data, header, partitions = None):
    ruidList = data.map(process_row123(header,
                               key =['ns_vid']), partitions).\
                filter(lambda x: x is not None).reduceByKey(lambda x,y: x if y in x.split(":") else x+":"+y)
    return ruidList
ruids = getRUID(data, header,10)

#nidsWithRuids = ruids
#nidsWithRuids = ruids.filter(lambda a : a[1] != "")
#nidsWithOutRuids = ruids.filter(lambda a : a[1] == "") ############Remove comment for all processing

In [ ]:
## Below line only for multiple RUIDS for a nsvid
ruids123 = ruids.filter(lambda x : x if len(x[1].split(":"))>1 and "" in x[1].split(":") else None).filter(lambda x: x is not None)
#ruidsCounts = ruids.countByKey()
ruids123.take(100)

In [ ]:
def get_distinct_users():
    def inner_row(row):
        return len(row[1].split(":"))
    return inner_row
    

In [ ]:
ruids123Wcounts = ruids123.map(get_distinct_users())
ruids123Wcounts.sum()

In [ ]:
rdlist = ruids.map(get_distinct_users())
rdlist.sum()

In [ ]:
ruidsDF = sqlContext.createDataFrame(nidsWithRuids,['ruidWRD', 'ns_vidWRD'])
#ruidsDF.persist()
ruidsDF.count()

In [ ]:
ruidsDFgrpd = ruidsDF.groupBy(ruidsDF.ruidWRD)
#ruidsDFFilt = ruidsDF.where("ruidWRD='8eb54a12-a758-4061-b6ef-f285dd248628'").collect()
#ruidsDFFilt = ruidsDF.filter(ruidsDF.ruidWRD=="8eb54a12-a758-4061-b6ef-f285dd248628").collect()

In [ ]:
ruidsDFgrpd

In [ ]:
ruidsDFgrpdgrp.

In [ ]:
noRuidsDF = sqlContext.createDataFrame(nidsWithOutRuids,['ns_vidWORD', 'ruidWORD'])
noRuidsDFgrpByVID = noRuidsDF.groupBy(noRuidsDF.ns_vidWORD)
noRuidsDFgrpByVIDcount = noRuidsDFgrpByVID.count()

In [ ]:
noRuidsDFgrpByVIDcount.take(10)

In [ ]:
joinedDF = noRuidsDF.join(ruidsDF,ruidsDF.ns_vidWRD==noRuidsDF.ns_vidWORD,"inner")

In [ ]:
from pyspark import SparkConf, SparkContext

# In Jupyter you have to stop the current context first
sc.stop()

# Create new config
conf = (SparkConf().set("spark.driver.maxResultSize", "2g"))

# Create new context
sc = SparkContext(conf=conf)
joinedDF

In [ ]:
df2.select("ns_vidWRD")

In [ ]:
ruidsDFGrouped = ruidsDF.groupBy("ns_vid")
x = ruidsDFGrouped.count()
x.count()
#ruidsTable = sqlContext.registerDataFrameAsTable(ruidsDF, "ruidtab")
#ruidsTable
#valuesOnly = ruids.values()
#valuesOnly.take(10)
#def printSplitted():
#    splitted = row[0].split(':')
#    splittedSet = set(splitted)
#    return splittedSet
#vals = valuesOnly.map(printSplitted())
#vals.take(100)

In [ ]:
x=['d6e8f457-d0f6-4034-ae09-0455c3388f5d:d6e8f457-d0f6-4034-ae09-0455c3388f5d',
 '13918c42-c153-448b-9bc4-2497043e1761:13918c42-c153-448b-9bc4-2497043e1761',
 '1c065a72-3a7d-4238-8931-7e53107212a9:1c065a72-3a7d-4238-8931-7e53107212a9',
 'd521b07e-c8ac-49b5-9931-086190b453bc:d521b07e-c8ac-49b5-9931-086190b453bc',
 'f6dcb7e4-379a-499c-81ec-55f0e0f3cb60:f6dcb7e4-379a-499c-81ec-55f0e0f3cb60',
 'cfc8835d-05f4-4cb4-bd84-9c4b073fafc5:cfc8835d-05f4-4cb4-bd84-9c4b073fafc5',
 'aae1a835-1063-494d-87fe-78f65c0c8c02:aae1a835-1063-494d-87fe-78f65c0c8c02',
 'be045942-2b4c-4fdc-ac01-ca0850f7e18a:be045942-2b4c-4fdc-ac01-ca0850f7e18a:be045942-2b4c-4fdc-ac01-ca0850f7e18a',
 '4bb9aeef-7c78-4c03-a0a5-4f2bbb018e5b:4bb9aeef-7c78-4c03-a0a5-4f2bbb018e5b:4bb9aeef-7c78-4c03-a0a5-4f2bbb018e5b',
 '587b0acc-49ec-42b1-a58f-0d4a4af278be:587b0acc-49ec-42b1-a58f-0d4a4af278be']
for a in x:
    splitted = a.split(':')
    splittedSet = set(splitted)
    print(splittedSet)
        

In [ ]:
row = data.take(100)[4]
key=['ns_vid','_site']
stripped = row.strip().split('\t')
row = row.split('\t')
row = dict(zip(header.value[:len(row)], row))
keys = tuple(row.get(k, None) for k in key)
keys = keys[0] if len(keys) == 1 else keys
if row['ruid'] is None or row['ruid'] != '':
    if row['_site'] == 'tasteofhome':
        print (keys, row['agent'])

In [ ]:
input_paths = [S3_FIRST_PASS_PATTERN.format(log_date='2016-04-21',site='rd'),S3_FIRST_PASS_PATTERN.format(log_date='2016-04-22',site='rd')]
vsessions = sqlContext.read.parquet(*input_paths).rdd
vsessions_group = vsessions.map(lambda x: ((x.id, x.site,x.ruid), \
                [eval_dict(d) for d in x.sessions]), True)
vsessions_comb = vsessions_group.reduceByKey(combineLists)
vsessions_comb.take(3)

In [ ]:
fmt = '%Y-%m-%d %H:%M:%S'
d1 = datetime.strptime('2010-01-01 17:31:22', fmt)
d2 = datetime.strptime('2010-01-01 21:31:22', fmt)
c = (d2-d1)
d = c.total_seconds()/60
d

In [ ]:
def filterUsers(recur):
    def innerfilterUsers(row):
        sessions = row[1]
        session_dates = []
        for s in sessions:
            if len(session_dates) <= recur:
                session_dates.append(s['session_start_time'])
        return row if len(session_dates) >= recur else None
    return innerfilterUsers
vsessions_filtered = vsessions_comb.map(filterUsers(2)).filter(lambda x : x is not None)
vsessions_filtered.take(2)

In [ ]:
data_S3_FIRST_PASS_PATTERN = sqlCtx.read.parquet(S3_FIRST_PASS_PATTERN.format(log_date='2016-05-09',site='rd'))
RD_FP_DATA = data_S3_FIRST_PASS_PATTERN.rdd
def getUrlPath(row):
    parsed = urlparse('//'+row)
    return (parsed.netloc,1)
def getSearched(row):
    return (row,1)
def get_passed_tags_data(tag):
    def inner_get_users_multiple_visits(row):   
        sessionData=row[2]
        tag_array = []
        for oneSession in sessionData:
            if tag in oneSession.keys():
                #paths = ast.literal_eval(oneSession[tag])
                #for path_time in paths:
                    #if (isinstance(path_time[1],int)):
                usragn = parse(oneSession[tag])
                tag_array.append(usragn.browser.family)
        return tag_array
    return inner_get_users_multiple_visits

def get_passed_tags_data_from_value_details(tag):
    def inner_get_users_multiple_visits(row):   
        sessionData=row[2]
        tag_array = []
        for oneSession in sessionData:
            if 'value_details' in oneSession.keys():
                value_details = ast.literal_eval(oneSession['value_details'])
                #paths = ast.literal_eval(oneSession[tag])
                #for path_time in paths:
                    #if (isinstance(path_time[1],int)):
                if tag in value_details.keys():
                    #for key,value in value_details[tag].items():
                    tag_array.extend(value_details[tag].keys())
        return tag_array
    return inner_get_users_multiple_visits


all_tag_data = RD_FP_DATA.flatMap(get_passed_tags_data_from_value_details('ns_event'))
#all_tag_data_distinct = all_tag_data.distinct()
#all_tag_data_distinct.coalesce(1).saveAsTextFile("/home/analytics/testdata/rd/sampleValues/categories")
all_tag_data1 = all_tag_data.map(getSearched)
#all_tag_data.take(20)
#parsedtimes = all_tag_data.map(getUrlPath)
all_tag_data1.reduceByKey(add).sortBy(lambda x: -x[1]).collect()

In [ ]:
sa = SessionAggregator(datetime(2016, 4, 20), 
        datetime(2016, 4, 30))
from inspect import getmembers
header = ['visitor_id', 'site']
for member in getmembers(sa):
    if callable(member[1]) and not member[0].startswith('_'):
        header.append(member[0])
header
header = sc.broadcast(header)
header.value

In [ ]:
sa = SessionAggregator(datetime(2016, 5, 23), 
        datetime(2016, 7, 3))
from inspect import getmembers
header = ['visitor_id', 'site']
for member in getmembers(sa):
    if callable(member[1]) and not member[0].startswith('_'):
        header.append(member[0])
header
header = sc.broadcast(header)

s3_visitor_data = sc.textFile(S3_VISITOR_FINAL_PATTERN.format(site="rd",start="2016-05-23",end="2016-07-03",source="parquet"))

def getMappedData(header):
    def innerGetMappedData(row):
        splittedRow = row.split('\t')
        row = dict(zip(header.value[:len(splittedRow)], splittedRow))
        return row
    return innerGetMappedData

def mapData(visitorFinalPassData):
    vData = visitorFinalPassData.map(getMappedData(header)).filter(lambda x: x is not None)
    return vData
visitFinalData = mapData(s3_visitor_data)

#visitFinalData=visitFinalData.filter(lambda x: x['twitter_content_terms']!='')
visitFinalData.count()